In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
import chromedriver_autoinstaller
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [8]:
# 미국 url = "https://findbusiness.us/eng/city/new-york-NY"
url = "https://gb.bizin.eu/eng/cat/fast-foods"

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  # Check chromedriver version
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito") # incognito mode
chrome_options.add_argument("--no-sandbox") # Prevent access to resources
chrome_options.add_argument("--disable-setuid-sandbox") # Prevent chrome crashes
chrome_options.add_argument("--disable-dev-shm-usage") # Prevent out of memory errors
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)   

first = 0
for num in range(1, 100):
    if num != 1: driver.get(f"{url}?p={num}")
    else: driver.get(url)
    try: org_lst = driver.find_element(By.XPATH, "/html/body/div/main/div/div[6]/div[1]")
    except:
        try: org_lst = driver.find_element(By.XPATH, "/html/body/div[2]/main/div/div[6]/div[1]")
        except:
            try: org_lst = driver.find_element(By.XPATH, "/html/body/div[3]/main/div/div[6]/div[1]")
            except: pass
    href_lst = {}
    for href in org_lst.find_elements(By.TAG_NAME, "div"):
        try:
            href_lst[(href.find_element(By.TAG_NAME, "a").text)] = href.find_element(By.TAG_NAME, "a").get_attribute("href")
        except: pass
    href_lst = {key:value for key, value in href_lst.items() if "eng" in value}


    for name, idx in tqdm(href_lst.items()):
        try:
            driver.get(idx)
            company_name = name
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            if first == 0:
                df = pd.read_html(str(soup.find('table')))[0]
                columns = df.transpose().iloc[0]
                df = df.T
                df.columns = columns
                df = df.drop(0, axis=0)
                df["name"] = company_name
                first += 1
            else:
                append_df = pd.read_html(str(soup.find('table')))[0]
                append_df_columns = append_df.transpose().iloc[0]
                append_df = append_df.T
                append_df.columns = list(append_df_columns)
                append_df = append_df.drop(0, axis=0)
                append_df["name"] = company_name
                df = df.append(append_df)
        except : pass

  7%|▋         | 1/14 [00:01<00:23,  1.80s/it]C:\Users\user\AppData\Local\Temp\ipykernel_43036\3588073693.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(append_df)
 14%|█▍        | 2/14 [00:03<00:18,  1.58s/it]C:\Users\user\AppData\Local\Temp\ipykernel_43036\3588073693.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(append_df)
 21%|██▏       | 3/14 [00:04<00:15,  1.39s/it]C:\Users\user\AppData\Local\Temp\ipykernel_43036\3588073693.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(append_df)
 29%|██▊       | 4/14 [00:06<00:15,  1.54s/it]C:\Users\user\AppData\Local\Temp\ipykernel_43036\3588073693.py:52: FutureWarning: The frame.append method is deprecated and will be 

In [9]:
df.to_csv("영국(bizin).csv", index=False)

In [10]:
df

,address,E.mail,Webpage,Phone number,Fax,Postal Code,City,Country,VCARD QR,name,Mobile,Facebook
1,"RG21 7DE,7 New Street, Basingstoke, Basingstok...",-,www.topspizza.com,+441256351111,-,RG21 7DE,"Hampshire, Basingstoke",Great Britain,NaN,Tops Pizza,NaN,NaN
1,"S71 2QR,Unit 2, 4 Castle Close, Monk Bretton, ...",-,www.woodys-chippies.co.uk,+441226247177,-,S71 2QR,Barnsley,Great Britain,NaN,"Woody""s Fish and Chips",NaN,NaN
1,"CB1 3LP,178 Mill Road, Cambridge, Cambridge , ...",-,www.perfectpizza.co.uk,+441223410800,+441223414090,CB1 3LP,"Cambridgeshire, Cambridge",Great Britain,NaN,Perfect Pizza,NaN,NaN
1,"SE22 8HU,121 Lordship Lane, London , London Gr...",-,www.gbkinfo.com,+442086939307,+442086939487,SE22 8HU,London,Great Britain,NaN,Gourmet Burger Kitchen,NaN,NaN
1,"NP26 4BQ,12 Newport Road, Caldicot, Caldicot ,...",-,www.thebaguetteshop.co.uk,+441291424249,+441291424249,NP26 4BQ,"Gwent, Caldicot",Great Britain,NaN,The Baguette Shop,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1,"BD2 1PT,219 Wrose Road, Bradford, West Ayton ,...",-,NaN,+441274587373,-,BD2 1PT,"West Yorkshire, West Ayton",Great Britain,NaN,Wrose Balti,NaN,NaN
1,"SY6 6BY,63 High Street, Church Stretton, Churc...",-,NaN,+441694723317,-,SY6 6BY,"Shropshire, Church Stretton",Great Britain,NaN,Ying Wah,NaN,NaN
1,"DG16 5AQ,109 Central Avenue, Gretna, Gretna , ...",-,NaN,+441461338269,-,DG16 5AQ,"Dumfriesshire, Gretna",Great Britain,NaN,Sark Services,NaN,NaN
1,"SA15 1DJ,44 Murray Street, Llanelli, Llanelli ...",-,NaN,+441554741791,-,SA15 1DJ,"Dyfed, Llanelli",Great Britain,NaN,The Oyster Fishbar,NaN,NaN


In [25]:
bizin_lst

['독일(bizin).csv',
 '미국(bizin).csv',
 '스페인(bizin).csv',
 '영국(bizin).csv',
 '이탈리아(bizin).csv',
 '프랑스(bizin).csv',
 '호주(bizin).csv']